<a href="https://colab.research.google.com/github/gutembergue-martins/sistema_de_assistencia_virtual/blob/main/Sistema_assistencia_virtual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Criando um sistema de assistência virtual do zero
Código Python

In [ ]:
# Verifica e instala dependências necessárias
try:
    import pyttsx3
    import webbrowser
    import speech_recognition as sr
    from google.cloud import speech
except ImportError:
    print("Instalando dependências necessárias...")
    os.system("pip install pyttsx3 webbrowser SpeechRecognition google-cloud-speech pyaudio")
    import pyttsx3
    import webbrowser
    import speech_recognition as sr
    from google.cloud import speech

# Inicializa o motor de texto-para-fala
engine = pyttsx3.init()

# Inicializa o reconhecedor de fala
recognizer = sr.Recognizer()

# Inicializa o cliente da API do Google Speech-to-Text
client = speech.SpeechClient()

# Função para transformar texto em áudio
def text_to_speech(text):
    """
    Converte texto em fala usando o motor pyttsx3.

    Args:
        text (str): Texto a ser convertido em fala.
    """
    engine.say(text)
    engine.runAndWait()

# Função para capturar áudio e converter em texto
def speech_to_text_google():
    """
    Captura áudio do microfone e converte em texto usando a API do Google Speech-to-Text.

    Returns:
        str: Texto reconhecido a partir do áudio.
    """
    print("Diga algo...")
    text_to_speech("Estou ouvindo...")

    with sr.Microphone() as source:
        recognizer.adjust_for_ambient_noise(source, duration=1)  # Redução de ruído
        audio = recognizer.listen(source)  # Captura do áudio

    try:
        response = client.recognize(
            config=speech.RecognitionConfig(
                encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
                sample_rate_hertz=44100,
                language_code="pt-BR",
                enable_automatic_punctuation=True,  # Adiciona pontuação automática
                model="latest_long"  # Usa o modelo mais atualizado
            ),
            audio=speech.RecognitionAudio(content=audio.get_wav_data()),
        )

        for result in response.results:
            recognized_text = result.alternatives[0].transcript.lower().strip()
            print(f"Você disse: {recognized_text}")
            return recognized_text

    except Exception as e:
        print(f"Erro no reconhecimento de voz: {e}")
        text_to_speech("Não consegui entender. Tente novamente.")

    return None

# Função para abrir qualquer página da web
def open_website(command):
    """
    Abre uma página da web com base no comando de voz.

    Args:
        command (str): Comando de voz contendo o nome do site.
    """
    site_name = command.replace("abrir", "").strip()
    if not site_name:
        text_to_speech("Por favor, diga o nome do site.")
        return

    # Converte para URL válida
    if "." not in site_name:
        site_name = site_name.replace(" ", "") + ".com.br"

    url = f"https://www.{site_name}"
    webbrowser.open(url)
    text_to_speech(f"Abrindo {site_name}.")

# Funções Específicas
def search_wikipedia(query):
    """
    Abre a Wikipedia e pesquisa pelo termo fornecido.

    Args:
        query (str): Termo de pesquisa.
    """
    url = f"https://pt.wikipedia.org/wiki/{query.replace(' ', '_')}"
    webbrowser.open(url)
    text_to_speech(f"Abrindo a Wikipedia para {query}.")

def open_youtube():
    """Abre o YouTube no navegador padrão."""
    webbrowser.open("https://www.youtube.com")
    text_to_speech("Abrindo o YouTube.")

def open_google_maps():
    """Abre o Google Maps no navegador padrão."""
    webbrowser.open("https://www.google.com/maps")
    text_to_speech("Abrindo o Google Maps.")

# Função principal do assistente virtual
def virtual_assistant():
    """
    Função principal que gerencia a interação com o assistente virtual.
    """
    text_to_speech("Olá! Como posso ajudar você hoje?")

    while True:
        command = speech_to_text_google()
        if not command:
            continue

        if "wiki" in command:
            query = command.replace("wiki", "").strip()
            search_wikipedia(query if query else "Página principal")
        elif "youtube" in command:
            open_youtube()
        elif "google maps" in command:
            open_google_maps()
        elif "abrir" in command:
            open_website(command)
        elif "sair" in command or "parar" in command:
            text_to_speech("Até logo!")
            break
        else:
            text_to_speech("Desculpe, não entendi o comando. Tente novamente.")

# Executa o assistente virtual
if __name__ == "__main__":
    virtual_assistant()